In [1]:
import scipy as sp
import numpy as np
import sympy as sy
import scipy.integrate as spi
from sympy.abc import symbols
import scipy.optimize as spo
from sympy.utilities.lambdify import lambdify 
import itertools as it
import time
import matplotlib.pyplot as plt
import iminuit as im

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [13]:
t,x,y, z, w, h, a,b,c,d, e, f, g,h = symbols('t,x,y, z, w, h, a,b,c,d, e,f,g,h', real = True)

In [4]:
#construct homotopy
def H(t, G, F, gamma):
    return [(1 - t)*G[i] + gamma*t*F[i] for i in range(len(G))]

#for degree 3 polynomial, do we need to generalise for more than one variable??
def G(x_array):
    func_listG = [i**3 - 1 for i in x_array]
    return func_listG

#construct inital starting systems for different number of starting systems
def F3(x_array):
    return [x_array[0]**3 - 6*x_array[1] + np.pi*x_array[2]**2, x_array[1]**3 + x_array[2]**2 + 8*x_array[0], x_array[2]**3 \
            + x_array[0]*x_array[1]*x_array[2] + np.pi]

def F4(x_array):
    return [x_array[0]**3 - 6*x_array[1] + np.pi*x_array[2]**2 + np.pi*x_array[3]**2, x_array[1]**3 + x_array[2]**2 + 8*x_array[0], x_array[2]**3 \
            + x_array[0]*x_array[1]*x_array[2] + np.pi, x_array[3]**3 - np.pi]

def F2predictortest(x_array):
    return [x_array[0] - 10, x_array[1] - 10]

def F2(x_array):
    return [x_array[0]**3 - 6*x_array[1] + np.pi, x_array[1]**3 + 8*x_array[0] + 76]

def F2Easy(x_array):
    return [x_array[0]**2  + np.pi, x_array[1]**2 - 76] 

def F1(x_array):
    return [x_array[0]**3 + np.pi + x_array[0]**2 - x_array[0]]

def gamma_generate():
    real = np.random.rand()
    im = np.sqrt(1- real**2)
    return real + im*1j

def G_roots_find(n):
    root_list = [1, np.exp(1j*2*np.pi/3), np.exp(1j*2*np.pi*2/3)]
    return [i for i in it.product(root_list, repeat = n)]

def split_function(func, expansion_array, expansion_variables):
    
    im_real_func = func(expansion_array)
    full_array = sum([abs(sy.re(i_re)) for i_re in im_real_func] + [abs(sy.im(i_im)) for i_im in im_real_func])
    
    full_func = lambdify(expansion_variables, full_array)
    
    return full_func


In [46]:
def Homotopy_Continuation(t, x_array, F, N, expansion_array, expansion_variables, tolerance = 1e-10, tolerance_zero = 1e-10, ratio_tolerance = 1e-5,\
                          N_ratio_tolerance = 50, debug = False, corrector_Newtons = True):
    
    time_start = time.time()
    
    #count the number of roots
    num = 0
    delta_t = 1/N
    n = len(x_array)
    gamma = gamma_generate()
    gamma= 0.6 + 0.8j
    G_roots = G_roots_find(n)
    
    #construct homotopy
    H_func = H(t, G(x_array), F(x_array), gamma)
    
    #first derivative of H
    H_diff_x = sy.Matrix([[H_func[i].diff(x_array[j]) for i in range(len(x_array))] for j in range(len(x_array))])
    

    determinant_H = H_diff_x.det()
    
    #derivative with respect to t
    H_diff_t = sy.Matrix([H_func[i].diff(t) for i in range(len(x_array))])
    
    #check determinant is not zero so can invert
    if abs(determinant_H) == 0:
        raise TypeError('The determinant of H is zero!')
    
    #function of determinant H
    det_H = lambdify((t, x_array), determinant_H)
    
    H_diff_x_inv = H_diff_x**-1
    H_diff_inv = -H_diff_x_inv*H_diff_t
    
    H_Hdiffprime = H_diff_x_inv*sy.Matrix(H_func)
    
    H_Hdiffprime = lambdify((t, x_array), [H_Hdiffprime[i] for i in range(len(H_Hdiffprime))])
        
    H_prime = lambdify((t, x_array), [H_diff_inv[i] for i in range(len(H_diff_inv))])
    
    Hprime_x = lambdify((x_array,t), [H_diff_x[i] for i in range(len(H_diff_x))])    
    H_func_1d = lambdify([x_array,t], H_func)
    

    x_old_arrays = []
    x_olds = []
    
    #run for all roots of starting system
    for x_old in G_roots:
        x_old_array = []
        num += 1
            
        t_n = 0
        
        #run for all steps starting at t=0 ending at t=1
        while round(t_n,5) < 1:

            x_old_array.append(x_old)
            t_old = t_n
            t_n += delta_t
            
            if n == 1:
                sol = spi.solve_ivp(H_prime, (t_old, t_n), x_old)
                sol_x = sol.y[-1][-1]
                
                H_func_1 = lambdify((x,t), H_func_1d([x], t)[0])
                Hprime_x_1 = lambdify((x,t), Hprime_x([x], t)[0])

                #x_old = [spo.newton(H_func_1, sol_x, fprime = Hprime_x_1, args=(t_n, ))]
            
            else:
                if abs(det_H(t_n, x_old)) < tolerance_zero:
                    raise TypeError('The determinant of H is zero!')
                
                #perform RK4 method
                x_old_predictor=x_old
                sol = spi.solve_ivp(H_prime, (t_old, t_n), x_old)
                sol_x = sol.y[:,-1]
                   
            #newton's method
            x_old = sol_x
            ratio = np.full(n, 1)
            N_ratio = 0
            delta = np.full(n, ratio_tolerance)
            
            if corrector_Newtons is True:
                
                time_newtons_start = time.time()
                #tolerance criteria for step size in Newton's Method
                while max(ratio) > ratio_tolerance and N_ratio < N_ratio_tolerance:
                    if abs(det_H(t_n, x_old)) < tolerance_zero:
                        raise TypeError('The determinant of H is zero!')
                    x_old_intermediate = x_old - H_Hdiffprime(t_n, x_old)
                    delta_old = delta
                    delta = abs(x_old_intermediate - x_old)
                    ratio = [delta[j]/(delta_old[j] + 1e-10) for j in range(n)]
                    x_old = x_old_intermediate
                    N_ratio += 1
                    #x_old = spo.newton(H_func, sol_x, fprime = Hprime_x, args=(t_n, ))
                    
                    time_newtons_end = time.time()
                if debug:

                    print('Time for Newton: {}'.format(time_newtons_end - time_newtons_start))
                    
            else:
                time_minuit_start = time.time()
                #iminuit
                H_func_t = H(t_n, G(expansion_array), F(expansion_array), gamma)

                #split real and imaginary
                H_im_real_array = sum([abs(sy.re(i_re)) for i_re in H_func_t] + [abs(sy.im(i_im)) for i_im in H_func_t])

                H_func_t_combined = lambdify([expansion_variables], H_im_real_array)

                x_old_re_im = []

                #split x_old to re and im
                for i in range(n):
                    x_old_re_im.append(np.real(x_old[i]))
                    x_old_re_im.append(np.imag(x_old[i]))
                    
                string_variables = [str(j) for j in expansion_variables]
                #call iminuit function

                m = im.Minuit(H_func_t_combined, x_old_re_im, forced_parameters= string_variables, use_array_call=True)
                m.migrad()

                x_old_im_re_vals = m.values
                
                x_old = [x_old_im_re_vals[j] + 1j*x_old_im_re_vals[j+1] for j in range(0, 2*n, 2)]
                time_minuit_end = time.time()
                
                if debug:
                    print('Time for Minuit: {}'.format(time_minuit_end - time_minuit_start))

        #check root is found
        remainder = list(map(abs, F(x_old))) 

        if max(remainder) < tolerance:
            x_old = [x_old[i].real if abs(x_old[i].imag) < tolerance_zero else x_old[i] for i in range(len(x_old))]

            print('Root {}, \n{}, \nAccuracy {}!!\n'.format(num, x_old, remainder))
            x_olds.append(x_old)
            x_old_arrays.append(x_old_array)

    time_end = time.time()
    
    print('It took {} s to run!'.format(time_end - time_start))
    
    return x_olds, x_old_arrays

In [40]:
x_sols, x_sols_arrays = Homotopy_Continuation(t, [x,y,z], F3, N=10, expansion_array= [a + 1j*b, c + 1j*d, e + 1j*f], expansion_variables= [a,b, c,d,e,f], \
                                              tolerance = 1, N_ratio_tolerance= 100, ratio_tolerance= 1e-5,debug=False, corrector_Newtons = True)

Root 1, 
[(-2.3255510470269294-1.0006333453462228j), (-1.038838253209918-2.5287848976712097j), (0.02072989957897064+0.45185182981774386j)], 
Accuracy [0.001988571530193719, 0.002459507345345748, 0.00042840994878384217]!!

Root 2, 
[(2.3686628518079975-0.9451797094263983j), (1.0500569860841757-2.5135831710570997j), (0.006021392774093052-0.4518231752930794j)], 
Accuracy [0.0003243291381793329, 0.0003967763825963619, 6.679432875996029e-05]!!

Root 3, 
[(2.368680267906502+0.9451609249568809j), (1.0500331328166217+2.513584450571871j), (0.006015911795646599+0.4518398250465659j)], 
Accuracy [0.0003458998490837199, 0.00042316264613695724, 7.123397728517817e-05]!!

Root 4, 
[(-2.3255510470269294-1.0006333453462228j), (-1.038838253209918-2.5287848976712097j), (0.02072989957897064+0.45185182981774386j)], 
Accuracy [0.001988571530193719, 0.002459507345345748, 0.00042840994878384217]!!

Root 5, 
[(-2.3255510470269294-1.0006333453462228j), (-1.038838253209918-2.5287848976712097j), (0.020729899578970

In [ ]:
x_sols, x_sols_arrays = Homotopy_Contin.3uation(t, [x,y], F2, N=1000, expansion_array= [a + 1j*b, c + 1j*d], expansion_variables= [a,b, c,d], \
                                              tolerance = 1000, N_ratio_tolerance= 100, ratio_tolerance= 1e-5,debug=False, corrector_Newtons = False)

Root 1, 
[(-2.947118202403947-4.5654877466220506e-07j), (-3.7426053839971534+8.3337260988158e-07j)], 
Accuracy [2.1327895626301927e-05, 4.010711770132734e-05]!!

Root 2, 
[(-2.947118202403947-4.5654877466220506e-07j), (-3.7426053839971534+8.3337260988158e-07j)], 
Accuracy [2.1327895626301927e-05, 4.010711770132734e-05]!!

Root 3, 
[(-2.947118202403947-4.5654877466220506e-07j), (-3.7426053839971534+8.3337260988158e-07j)], 
Accuracy [2.1327895626301927e-05, 4.010711770132734e-05]!!

Root 4, 
[(-2.947118202403947-4.5654877466220506e-07j), (-3.7426053839971534+8.3337260988158e-07j)], 
Accuracy [2.1327895626301927e-05, 4.010711770132734e-05]!!



In [ ]:
for i in range(0, 6, 2):
    print(i)

In [ ]:
H_func_1d = lambdify((x_array,t), H_func)

In [ ]:
def split_function(func, expansion_array, expansion_variables):
    
    im_real_func = func(expansion_array)
    full_array = sum([abs(sy.re(i_re)) for i_re in im_real_func] + [abs(sy.im(i_im)) for i_im in im_real_func])
    
    full_func = lambdify([expansion_variables], full_array)
    
    return full_func

In [ ]:
F2real = split_function(F2Easy, [a + 1j*b, c + 1j*d], [a,b,c,d])

In [ ]:
F2real([a,b,c,d])

In [ ]:
hhh = abs(x**2 - 8 + y) + abs(y**2 - 16)

In [ ]:
hhh = lambdify((x, y), hhh

In [ ]:
ggg = ['a','b','c','d']
m = im.Minuit(F2real, [1,2,3,4], forced_parameters= ggg, use_array_call=True)
m.migrad()
new_vals = m.values

In [ ]:
new_vals[1]

In [ ]:
def tryh(x, y, z, g):
    return x+y+z+g

hi = x**2 + y**3 + b
array_y = [x,y]

In [ ]:
argg = [2, 3,4]

In [ ]:
tryh(1, *argg)

In [ ]:
p = 5.66  + 87j

In [ ]:
np.imag(p)

In [ ]:
tryggg = lambdify([x, y, b], hi)

In [ ]:
tryggg(1,2, 1)